Youtube link: https://www.youtube.com/watch?v=cXHvC_FGx24

_____

# Objective: we want to minimize $x_{1}x_{4}(x_{1}+x_{2}+x_{3})+x_{3}$

# Constaints:

## 1. $x_{1}x_{2}x_{3}x_{4} \geq 25$

## 2. $x_{1}^{2} + x_{2}^{2} + x_{3}^{2} + x_{4}^{2} = 40$

## 3. $1 \leq x_{i} \leq 5$ for $i \in \{1,2,3,4\}$

# Initial condition: $X_{0} = (1,5,5,1)$

____

### Before we start using `scipy` to minimize this, let's start by using only integers, and using `pandas`

In [1]:
import pandas as pd
import numpy as np

- We create a dataframe with all possible values for the $x_{i}$s
    - We know each $x_{i}$ is between 1 and 5 because of condition 3

In [2]:
array = np.arange(1, 6)
x1, x2, x3, x4 = np.meshgrid(array, array, array, array)

In [3]:
grid = np.c_[x1.ravel(),
            x2.ravel(),
            x3.ravel(),
            x4.ravel()]

In [4]:
cols = ['x1','x2','x3','x4']
df = pd.DataFrame(grid, columns = cols)

In [5]:
df.head()

,x1,x2,x3,x4
0,1,1,1,1
1,1,1,1,2
2,1,1,1,3
3,1,1,1,4
4,1,1,1,5


- Now, we can apply condition 1

In [6]:
df['condition 1'] = df[cols].product(axis=1)>=25

In [7]:
df.head()

,x1,x2,x3,x4,condition 1
0,1,1,1,1,False
1,1,1,1,2,False
2,1,1,1,3,False
3,1,1,1,4,False
4,1,1,1,5,False


- And finally, condition 2
    - Recall, we applied condition 3 when we created the dataframe

In [8]:
df['condition 2'] = (df[cols]**2).sum(axis=1)==40

- Now, we only select the rows where the two conditions are satisfied

In [9]:
df = df[(df['condition 1'])&(df['condition 2'])]

In [10]:
df.head()

,x1,x2,x3,x4,condition 1,condition 2
168,2,2,4,4,True,True
208,4,2,2,4,True,True
216,4,2,4,2,True,True
408,2,4,2,4,True,True
416,2,4,4,2,True,True


- Now, we apply our function we're trying to minimize

In [11]:
def objective_function(array):
    x1, x2, x3, x4 = array
    val = x1*x4*(x1+x2+x3) + x3
    return val

In [12]:
df['objective'] = df[cols].apply(objective_function, axis = 1)

In [15]:
df

,x1,x2,x3,x4,condition 1,condition 2,objective
168,2,2,4,4,True,True,68
208,4,2,2,4,True,True,130
216,4,2,4,2,True,True,84
408,2,4,2,4,True,True,66
416,2,4,4,2,True,True,44
456,4,4,2,2,True,True,82


- Now, we just need to find our minimum value in the objective column

In [13]:
df.loc[df['objective'].idxmin()]

x1                2
x2                4
x3                4
x4                2
condition 1    True
condition 2    True
objective        44
Name: 416, dtype: object

### Therefore, the values that satisfy our constraints and minimize our objective function are:

####  $[x_{1},x_{2},x_{3},x_{4}] = [2,4,4,2]$

____

### First, we note that we didn't use our initial conditions. It doesn't seem to have affected our answer

### Next, we note that we only used integers. Our dataframe would have been much bigger if we used decimals, etc. We also would have probably found a more optimal vector. We'll see this below...

____

## Let's do the same thing using `scipy.optimize`

In [31]:
from scipy.optimize import minimize

- We can use the same objective funciton defined above

- To feed our first constaint into the code, we translate: $x_{1}x_{2}x_{3}x_{4} \geq 25 \rightarrow x_{1}x_{2}x_{3}x_{4} -25 \geq 0$

In [32]:
def constraint1(array):
    return np.product(array) - 25

- Similarly, $x_{1}^{2} + x_{2}^{2} + x_{3}^{2} + x_{4}^{2} = 40 \rightarrow x_{1}^{2} + x_{2}^{2} + x_{3}^{2} + x_{4}^{2} - 40 = 0$

In [33]:
def constraint2(array):
    return np.sum(array**2) - 40

- Now, for condition 3, we define boundaries

In [34]:
bounds = [1,5]
bounds_array = np.array([bounds, bounds, bounds, bounds])

- We also need to define our initial condition

In [35]:
X0 = np.array([1,5,5,1])

- Now, we **need to define a dictionary that feeds details to our optimization function**

In [36]:
con1 = {'type':'ineq', 'fun':constraint1}
con2 = {'type':'eq', 'fun':constraint2}

- We create a list to feed these dictionaries into our function

In [37]:
constraints = [con1, con2]

### Finally, we have everything defined and can solve our objective function under our constraints

In [38]:
solution = minimize(objective_function, X0, method = 'SLSQP', bounds = bounds_array, constraints = constraints)

In [39]:
solution

     fun: 17.01401724556073
     jac: array([14.57227039,  1.37940764,  2.37940764,  9.56415081])
 message: 'Optimization terminated successfully.'
    nfev: 30
     nit: 5
    njev: 5
  status: 0
 success: True
       x: array([1.        , 4.74299607, 3.82115466, 1.37940764])

In [41]:
solution.x

array([1.        , 4.74299607, 3.82115466, 1.37940764])

# Therefore, the ideal set of values that satisfy our constraints and minimize our objective function are:

####  $[x_{1},x_{2},x_{3},x_{4}] = [1.0, 4.74299607, 3.82115466, 1.37940764]$